# Set up environment and load in data

In [ ]:
library(tidyverse)
library(here)
library(brms)
library(gridExtra)
theme_set(theme_bw())
helpers_path = paste0(here(),'/analysis/helpers/')
fig_out_path = paste0(here(), '/outputs/fig/')

Add columns that will be used for plots below.

In [ ]:
source(paste0(helpers_path, '01_clean_behavioral_data.R'))

data_yn_clean = data_yn_clean %>%
  mutate(correct = ifelse(possiblePayoff>reference & yesChosen == 1, 1, ifelse(possiblePayoff < reference & yesChosen == 0, 1, 0))) %>%
  mutate(type = ifelse(type == 1, "HT", "RE"),
         week = ifelse(week == 1, "Week 1", ifelse(week == 2, "Week 2", ifelse(week == 3, "Week 3", NA))),
         week = factor(week, levels = c("Week 1", "Week 2", "Week 3"))) %>%
  mutate(val_diff = possiblePayoff - reference,
         val_diff_bin = round(val_diff/50),
         val_diff_bin_str = paste0(val_diff_bin*50-25,":",val_diff_bin*50+25),
         val_diff_bin_str = factor(val_diff_bin_str, levels = c("-225:-175", "-175:-125", "-125:-75", "-75:-25", "-25:25", "25:75", "75:125", "125:175", "175:225")))

data_bc_clean = data_bc_clean %>%
  mutate(correct = ifelse(possiblePayoffleft>possiblePayoffright & leftChosen == 1, 1, ifelse(possiblePayoffleft<possiblePayoffright & leftChosen == 0, 1, 0))) %>%
  mutate(type = ifelse(typeLeft == 1, "HT", "RE"),
         week = ifelse(week == 1, "Week 1", ifelse(week == 2, "Week 2", ifelse(week == 3, "Week 3", NA))),
         week = factor(week, levels = c("Week 1", "Week 2", "Week 3"))) %>%
  mutate(val_diff = possiblePayoffleft - possiblePayoffright,
         val_diff_bin = round(val_diff/50),
         val_diff_bin_str = paste0(val_diff_bin*50-25,":",val_diff_bin*50+25),
         val_diff_bin_str = factor(val_diff_bin_str, levels = c("-275:-225","-225:-175", "-175:-125", "-125:-75", "-75:-25", "-25:25", "25:75", "75:125", "125:175", "175:225", "225:275"))) %>%
  filter(!is.na(val_diff_bin_str))

# Changes in HDDM pars

**[THIS ISN'T THE BEST APPROACH. TRY MODELING EACH SUBJECT HIERARCHICALLY INSTEAD]**

### Drift rate

In [ ]:
fn_draws = paste0(helpers_path, '/inputs/yn_ddm_dRegressions_draws.csv')

if(file.exists(fn_draws)){
  yn_rt_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_yn_clean$subnum)
  
  yn_ddm_draws = data_frame()
  
  for(i in 1:length(subnums)){
    cur_sub = subnums[i]
    
    cur_par_data = all_results_df %>% 
      filter(subnum == cur_sub & par_name == "d")
    
    # REDUCE AMOUNT OF DATA! 100 datapoints per bin
    cur_par_data = cur_par_data %>%
      group_by(type, day) %>%
      sample_n(100)
    
    cur_m <- brm(formula = estimate ~ day * type,
                 data= cur_par_data,
                 warmup = 500,
                 iter = 2500,
                 chains = 4,
                 init = "0",
                 cores = 4,
                 seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    yn_ddm_draws = rbind(yn_ddm_draws, cur_draws)
    
  }
  names(yn_ddm_draws) = gsub(":", ".", names(yn_ddm_draws))
  write.csv(yn_ddm_draws, fn_draws, row.names = F)
  
  rm(cur_par_data, cur_m, cur_draws)
}


In [ ]:
yn_par_summary = as.data.frame(yn_ddm_draws) %>%
  select("b_day" ,"b_typeRE", "b_day.typeRE", subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key = ifelse(key == "b_day", "Day", ifelse(key == "b_typeRE", "Type (RE)", ifelse(key == "b_day.typeRE", "Day * Type (RE)", NA))),
         key = factor(key, levels = c("Type (RE)", "Day", "Day * Type (RE)"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N") 

In [ ]:
plim = max(abs(yn_par_summary$l95), yn_par_summary$h95)

p = yn_par_summary %>%
  ggplot(aes(m, subnum)) +
  geom_vline(aes(xintercept = 0), color = "light gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_grid(.~key) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - d ~ Day * Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_continuous(breaks = c(0), limits = c(-plim, plim))

p
# ggsave(file=paste0(fig_out_path, 'yn_ddmD_parEsts.jpg'), p, height = 3, width = 7, units="in")


### Sigma

In [ ]:
fn_draws = paste0(helpers_path, '/inputs/yn_ddm_sigmaRegressions_draws.csv')

if(file.exists(fn_draws)){
  yn_rt_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_yn_clean$subnum)
  
  yn_ddm_draws = data_frame()
  
  for(i in 1:length(subnums)){
    cur_sub = subnums[i]
    
    cur_par_data = all_results_df %>% 
      filter(subnum == cur_sub & par_name == "sigma")
    
    # REDUCE AMOUNT OF DATA! 100 datapoints per bin
    cur_par_data = cur_par_data %>%
      group_by(type, day) %>%
      sample_n(100)
    
    cur_m <- brm(formula = estimate ~ day * type,
                 data= cur_par_data,
                 warmup = 500,
                 iter = 2500,
                 chains = 4,
                 init = "0",
                 cores = 4,
                 seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    yn_ddm_draws = rbind(yn_ddm_draws, cur_draws)
    
  }
  names(yn_ddm_draws) = gsub(":", ".", names(yn_ddm_draws))
  write.csv(yn_ddm_draws, fn_draws, row.names = F)
  
  rm(cur_par_data, cur_m, cur_draws)
}


In [ ]:
yn_par_summary = as.data.frame(yn_ddm_draws) %>%
  select("b_day" ,"b_typeRE", "b_day.typeRE", subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key = ifelse(key == "b_day", "Day", ifelse(key == "b_typeRE", "Type (RE)", ifelse(key == "b_day.typeRE", "Day * Type (RE)", NA))),
         key = factor(key, levels = c("Type (RE)", "Day", "Day * Type (RE)"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N") 

In [ ]:
plim = max(abs(yn_par_summary$l95), yn_par_summary$h95)

p = yn_par_summary %>%
  ggplot(aes(m, subnum)) +
  geom_vline(aes(xintercept = 0), color = "light gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_grid(.~key) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - sigma ~ Day * Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_continuous(breaks = c(0), limits = c(-plim, plim))

p
# ggsave(file=paste0(fig_out_path, 'yn_ddmSigma_parEsts.jpg'), p, height = 3, width = 7, units="in")


### Bias

In [ ]:
fn_draws = paste0(helpers_path, '/inputs/yn_ddm_biasRegressions_draws.csv')

if(file.exists(fn_draws)){
  yn_rt_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_yn_clean$subnum)
  
  yn_ddm_draws = data_frame()
  
  for(i in 1:length(subnums)){
    cur_sub = subnums[i]
    
    cur_par_data = all_results_df %>% 
      filter(subnum == cur_sub & par_name == "bias")
    
    # REDUCE AMOUNT OF DATA! 100 datapoints per bin
    cur_par_data = cur_par_data %>%
      group_by(type, day) %>%
      sample_n(100)
    
    cur_m <- brm(formula = estimate ~ day * type,
                 data= cur_par_data,
                 warmup = 500,
                 iter = 2500,
                 chains = 4,
                 init = "0",
                 cores = 4,
                 seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    yn_ddm_draws = rbind(yn_ddm_draws, cur_draws)
    
  }
  names(yn_ddm_draws) = gsub(":", ".", names(yn_ddm_draws))
  write.csv(yn_ddm_draws, fn_draws, row.names = F)
  
  rm(cur_par_data, cur_m, cur_draws)
}


In [ ]:
yn_par_summary = as.data.frame(yn_ddm_draws) %>%
  select("b_day" ,"b_typeRE", "b_day.typeRE", subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key = ifelse(key == "b_day", "Day", ifelse(key == "b_typeRE", "Type (RE)", ifelse(key == "b_day.typeRE", "Day * Type (RE)", NA))),
         key = factor(key, levels = c("Type (RE)", "Day", "Day * Type (RE)"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N") 

In [ ]:
plim = max(abs(yn_par_summary$l95), yn_par_summary$h95)

p = yn_par_summary %>%
  ggplot(aes(m, subnum)) +
  geom_vline(aes(xintercept = 0), color = "light gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_grid(.~key) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - bias ~ Day * Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_continuous(breaks = c(0), limits = c(-plim, plim))

p
# ggsave(file=paste0(fig_out_path, 'yn_ddmBias_parEsts.jpg'), p, height = 3, width = 7, units="in")


### NDT

In [ ]:
fn_draws = paste0(helpers_path, '/inputs/yn_ddm_ndtRegressions_draws.csv')

if(file.exists(fn_draws)){
  yn_rt_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_yn_clean$subnum)
  
  yn_ddm_draws = data_frame()
  
  for(i in 1:length(subnums)){
    cur_sub = subnums[i]
    
    cur_par_data = all_results_df %>% 
      filter(subnum == cur_sub & par_name == "ndt")
    
    # REDUCE AMOUNT OF DATA! 100 datapoints per bin
    cur_par_data = cur_par_data %>%
      group_by(type, day) %>%
      sample_n(100)
    
    cur_m <- brm(formula = estimate ~ day * type,
                 data= cur_par_data,
                 warmup = 500,
                 iter = 2500,
                 chains = 4,
                 init = "0",
                 cores = 4,
                 seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    yn_ddm_draws = rbind(yn_ddm_draws, cur_draws)
    
  }
  names(yn_ddm_draws) = gsub(":", ".", names(yn_ddm_draws))
  write.csv(yn_ddm_draws, fn_draws, row.names = F)
  
  rm(cur_par_data, cur_m, cur_draws)
}


In [ ]:
yn_par_summary = as.data.frame(yn_ddm_draws) %>%
  select("b_day" ,"b_typeRE", "b_day.typeRE", subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key = ifelse(key == "b_day", "Day", ifelse(key == "b_typeRE", "Type (RE)", ifelse(key == "b_day.typeRE", "Day * Type (RE)", NA))),
         key = factor(key, levels = c("Type (RE)", "Day", "Day * Type (RE)"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "Y/N") 

In [ ]:
plim = max(abs(yn_par_summary$l95), yn_par_summary$h95)

p = yn_par_summary %>%
  ggplot(aes(m, subnum)) +
  geom_vline(aes(xintercept = 0), color = "light gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_grid(.~key) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - ndt ~ Day * Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_continuous(breaks = c(0), limits = c(-plim, plim))

p
# ggsave(file=paste0(fig_out_path, 'yn_ddmNDT_parEsts.jpg'), p, height = 3, width = 7, units="in")
